In [1]:
import base64
import os
import json
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding, hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.backends import default_backend
from cryptography.fernet import Fernet

class PortfolioDataEncryptor:
    """
    A class for encrypting and decrypting personal data for a portfolio website.
    It provides methods for both personal data encryption before uploading to CDN
    and decryption methods that can be used in a website workflow.
    """
    
    def __init__(self):
        self.backend = default_backend()
    
    def generate_key(self, password, salt=None):
        """Generate a key using password and salt"""
        if salt is None:
            salt = os.urandom(16)
        
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,
            salt=salt,
            iterations=100000,
            backend=self.backend
        )
        key = kdf.derive(password.encode())
        return key, salt
    
    def encrypt_data(self, data, password):
        """
        Encrypt personal data using password
        
        Args:
            data (dict): Personal information to encrypt
            password (str): Password for encryption
            
        Returns:
            dict: Dictionary with encrypted data and metadata
        """
        # Convert data to JSON string
        data_str = json.dumps(data)
        
        # Generate key from password
        key, salt = self.generate_key(password)
        
        # Generate initialization vector
        iv = os.urandom(16)
        
        # Create cipher
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=self.backend)
        encryptor = cipher.encryptor()
        
        # Pad data
        padder = padding.PKCS7(algorithms.AES.block_size).padder()
        padded_data = padder.update(data_str.encode()) + padder.finalize()
        
        # Encrypt data
        encrypted_data = encryptor.update(padded_data) + encryptor.finalize()
        
        # Return encrypted data with metadata
        return {
            "encrypted_data": base64.b64encode(encrypted_data).decode('utf-8'),
            "iv": base64.b64encode(iv).decode('utf-8'),
            "salt": base64.b64encode(salt).decode('utf-8'),
            "version": "1.0"
        }
    
    def decrypt_data(self, encrypted_obj, password):
        """
        Decrypt personal data using password
        
        Args:
            encrypted_obj (dict): Dictionary with encrypted data and metadata
            password (str): Password for decryption
            
        Returns:
            dict: Decrypted personal information
        """
        # Extract encrypted data and metadata
        encrypted_data = base64.b64decode(encrypted_obj["encrypted_data"])
        iv = base64.b64decode(encrypted_obj["iv"])
        salt = base64.b64decode(encrypted_obj["salt"])
        
        # Generate key from password and salt
        key, _ = self.generate_key(password, salt)
        
        # Create cipher
        cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=self.backend)
        decryptor = cipher.decryptor()
        
        # Decrypt data
        decrypted_padded_data = decryptor.update(encrypted_data) + decryptor.finalize()
        
        # Unpad data
        unpadder = padding.PKCS7(algorithms.AES.block_size).unpadder()
        decrypted_data = unpadder.update(decrypted_padded_data) + unpadder.finalize()
        
        # Convert from JSON string back to dictionary
        return json.loads(decrypted_data.decode('utf-8'))
    
    def encrypt_to_file(self, data, password, file_path):
        """
        Encrypt personal data and save to file
        
        Args:
            data (dict): Personal information to encrypt
            password (str): Password for encryption
            file_path (str): Path to save encrypted file
        """
        encrypted_data = self.encrypt_data(data, password)
        with open(file_path, 'w') as f:
            json.dump(encrypted_data, f)
        
        print(f"Encrypted data saved to {file_path}")
    
    def decrypt_from_file(self, file_path, password):
        """
        Decrypt personal data from file
        
        Args:
            file_path (str): Path to encrypted file
            password (str): Password for decryption
            
        Returns:
            dict: Decrypted personal information
        """
        with open(file_path, 'r') as f:
            encrypted_obj = json.load(f)
        
        return self.decrypt_data(encrypted_obj, password)

    @staticmethod
    def generate_password_hash(password):
        """
        Generate a hash of the password that can be used for verification
        without storing the actual password
        
        Args:
            password (str): Password to hash
            
        Returns:
            str: Base64 encoded hash
        """
        digest = hashes.Hash(hashes.SHA256(), backend=default_backend())
        digest.update(password.encode())
        return base64.b64encode(digest.finalize()).decode('utf-8')
    
    @staticmethod
    def create_js_decryption_script():
        """
        Generate a JavaScript snippet that can be used for decryption in the browser
        
        Returns:
            str: JavaScript decryption function
        """
        return """
// Include this script in your portfolio website
// You'll need to include CryptoJS library: <script src="https://cdnjs.cloudflare.com/ajax/libs/crypto-js/4.1.1/crypto-js.min.js"></script>

async function decryptPortfolioData(encryptedObj, password) {
    // Get encrypted data and metadata
    const encryptedData = CryptoJS.enc.Base64.parse(encryptedObj.encrypted_data);
    const iv = CryptoJS.enc.Base64.parse(encryptedObj.iv);
    const salt = CryptoJS.enc.Base64.parse(encryptedObj.salt);
    
    // Generate key from password and salt
    const key = CryptoJS.PBKDF2(password, salt, {
        keySize: 256/32,
        iterations: 100000,
        hasher: CryptoJS.algo.SHA256
    });
    
    // Decrypt data
    const decrypted = CryptoJS.AES.decrypt(
        { ciphertext: encryptedData },
        key,
        { iv: iv, padding: CryptoJS.pad.Pkcs7, mode: CryptoJS.mode.CBC }
    );
    
    // Convert to string and parse JSON
    const decryptedStr = decrypted.toString(CryptoJS.enc.Utf8);
    return JSON.parse(decryptedStr);
}

// Example usage in portfolio site
async function loadPortfolioData() {
    try {
        // Fetch encrypted data from CDN
        const response = await fetch('https://your-cdn-url.com/encrypted-portfolio-data.json');
        const encryptedData = await response.json();
        
        // Get password from user or environment variable
        const password = prompt("Enter password to decrypt portfolio data:");
        
        // Decrypt data
        const portfolioData = await decryptPortfolioData(encryptedData, password);
        
        // Use the data to populate the portfolio (never store in localStorage or cookies)
        document.getElementById('name').textContent = portfolioData.name;
        document.getElementById('email').textContent = portfolioData.email;
        // ... other fields
        
    } catch (error) {
        console.error('Failed to load portfolio data:', error);
    }
}
"""

# Example usage
if __name__ == "__main__":
    # Create encryptor
    encryptor = PortfolioDataEncryptor()
    
    # Sample personal data
    personal_data = {
        "name": "John Doe",
        "email": "john@example.com",
        "phone": "+1234567890",
        "linkedin": "https://linkedin.com/in/johndoe",
        "github": "https://github.com/johndoe",
        "bio": "Full-stack developer with experience in React and Node.js",
        "experience": [
            {
                "position": "Senior Developer",
                "company": "Tech Corp",
                "duration": "2020-Present",
                "description": "Leading web development projects..."
            },
            {
                "position": "Junior Developer",
                "company": "Startup Inc",
                "duration": "2018-2020",
                "description": "Developed front-end components..."
            }
        ],
        "skills": ["JavaScript", "React", "Node.js", "Python", "Docker"]
    }
    
    # Set password for encryption
    password = "secure-password-example"
    
    # Encrypt data
    print("Encrypting personal data...")
    encrypted_data = encryptor.encrypt_data(personal_data, password)
    print("Data encrypted successfully.")
    print(f"Metadata: version={encrypted_data['version']}")
    
    # Save to file example (local usage before uploading to CDN)
    print("\nSaving encrypted data to file...")
    encryptor.encrypt_to_file(personal_data, password, "encrypted_portfolio_data.json")
    
    # Decrypt data (for testing)
    print("\nDecrypting data for verification...")
    decrypted_data = encryptor.decrypt_data(encrypted_data, password)
    print("Data decrypted successfully.")
    print(f"Decrypted name: {decrypted_data['name']}")
    print(f"Decrypted email: {decrypted_data['email']}")
    
    # Generate JavaScript decryption function
    print("\nGenerated JavaScript for browser decryption:")
    js_script = encryptor.create_js_decryption_script()
    print(f"JavaScript decryption function length: {len(js_script)} characters")
    print("This script can be included in the portfolio website to decrypt the data client-side.")

Encrypting personal data...
Data encrypted successfully.
Metadata: version=1.0

Saving encrypted data to file...
Encrypted data saved to encrypted_portfolio_data.json

Decrypting data for verification...
Data decrypted successfully.
Decrypted name: John Doe
Decrypted email: john@example.com

Generated JavaScript for browser decryption:
JavaScript decryption function length: 1956 characters
This script can be included in the portfolio website to decrypt the data client-side.
